In [1]:
import numpy as np
import pandas as pd
np.random.seed(8)
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from keras.models import Model
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback

import warnings
warnings.filterwarnings('ignore')

train = pd.read_csv('train.csv')
test_cm = pd.read_csv('test.csv')
test_lb = pd.read_csv('test_labels.csv')
test_all = pd.merge(test_cm, test_lb, on='id')
test = test_all[test_all['toxic'] != -1]

Using TensorFlow backend.


In [2]:
X_train = train["comment_text"].values
y_train = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
X_test = test["comment_text"].values
y_test = test[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values

In [3]:
vocab = 100000
maxlen = 200
embed_size = 300

In [4]:
t = text.Tokenizer(num_words=vocab)
t.fit_on_texts(list(X_train))

In [5]:
X_train = t.texts_to_sequences(X_train)
X_test = t.texts_to_sequences(X_test)
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)

In [6]:
EMBEDDING_FILE = 'crawl-300d-2M.vec'
embeddings_index = dict()
f = open(EMBEDDING_FILE)
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s pre-trained words' % len(embeddings_index))


Loaded 2000000 pre-trained words


In [7]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab, embed_size))
for word, i in t.word_index.items():
    if i >= vocab: 
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [8]:
def make_model(activation, filters, Sdroprate, droprate, layers):

    inp = Input(shape=(maxlen, ))
    x = Embedding(vocab, embed_size, weights=[embedding_matrix], input_length=maxlen, trainable=False)(inp)
    x = SpatialDropout1D(Sdroprate)(x)
    x = Reshape((maxlen, embed_size, 1))(x)

    conv_0 = Conv2D(filters, kernel_size=(2, embed_size), activation=activation)(x)
    maxpool_0 = MaxPool2D(pool_size=(maxlen - 1, 1))(conv_0)
    if layers == 2:   
        conv_1 = Conv2D(filters, kernel_size=(3, embed_size), activation=activation)(x)
        maxpool_1 = MaxPool2D(pool_size=(maxlen - 2, 1))(conv_1)
        y = Concatenate(axis=1)([maxpool_0, maxpool_1])
    elif layers == 3:
        conv_1 = Conv2D(filters, kernel_size=(3, embed_size), activation=activation)(x)
        maxpool_1 = MaxPool2D(pool_size=(maxlen - 2, 1))(conv_1)
        conv_2 = Conv2D(filters, kernel_size=(4, embed_size), activation=activation)(x)
        maxpool_2 = MaxPool2D(pool_size=(maxlen - 3, 1))(conv_2) 
        y = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2])
    elif layers == 4:
        conv_1 = Conv2D(filters, kernel_size=(3, embed_size), activation=activation)(x)
        maxpool_1 = MaxPool2D(pool_size=(maxlen - 2, 1))(conv_1)
        conv_2 = Conv2D(filters, kernel_size=(4, embed_size), activation=activation)(x)
        maxpool_2 = MaxPool2D(pool_size=(maxlen - 3, 1))(conv_2)
        conv_3 = Conv2D(filters, kernel_size=(5, embed_size), activation=activation)(x)
        maxpool_3 = MaxPool2D(pool_size=(maxlen - 4, 1))(conv_3)
        y = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2, maxpool_3])
    elif layers == 5:
        conv_1 = Conv2D(filters, kernel_size=(3, embed_size), activation=activation)(x)
        maxpool_1 = MaxPool2D(pool_size=(maxlen - 2, 1))(conv_1)
        conv_2 = Conv2D(filters, kernel_size=(4, embed_size), activation=activation)(x)
        maxpool_2 = MaxPool2D(pool_size=(maxlen - 3, 1))(conv_2)
        conv_3 = Conv2D(filters, kernel_size=(5, embed_size), activation=activation)(x)
        maxpool_3 = MaxPool2D(pool_size=(maxlen - 4, 1))(conv_3)
        conv_4 = Conv2D(filters, kernel_size=(6, embed_size), activation=activation)(x)
        maxpool_4 = MaxPool2D(pool_size=(maxlen - 5, 1))(conv_4)
        y = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2, maxpool_3, maxpool_4])
    else:
        conv_1 = Conv2D(filters, kernel_size=(3, embed_size), activation=activation)(x)
        maxpool_1 = MaxPool2D(pool_size=(maxlen - 2, 1))(conv_1)
        conv_2 = Conv2D(filters, kernel_size=(4, embed_size), activation=activation)(x)
        maxpool_2 = MaxPool2D(pool_size=(maxlen - 3, 1))(conv_2)
        conv_3 = Conv2D(filters, kernel_size=(5, embed_size), activation=activation)(x)
        maxpool_3 = MaxPool2D(pool_size=(maxlen - 4, 1))(conv_3)
        conv_4 = Conv2D(filters, kernel_size=(6, embed_size), activation=activation)(x)
        maxpool_4 = MaxPool2D(pool_size=(maxlen - 5, 1))(conv_4)
        conv_5 = Conv2D(filters, kernel_size=(7, embed_size), activation=activation)(x)
        maxpool_5 = MaxPool2D(pool_size=(maxlen - 6, 1))(conv_5)
        y = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2, maxpool_3, maxpool_4, maxpool_5])
    
    y = Flatten()(y)
    y = Dropout(droprate)(y)

    outp = Dense(6, activation="sigmoid")(y)

    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model
#model = make_model()
#model.summary()

In [9]:
x_trainS, x_val, y_trainS, y_val = train_test_split(x_train, y_train, train_size=0.90, random_state=8)

In [10]:
class RocAuc(Callback):

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.validation_data[0], verbose=0)
        score = roc_auc_score(self.validation_data[1], y_pred)
        print("\n roc_auc - epoch: %d - score: %.6f \n" % (epoch+1, score))

In [11]:
from keras.callbacks import ModelCheckpoint
batch_size=256
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.pre_trained.conv2.testscore.hdf5', verbose=1, save_best_only=True)
RocAuc = RocAuc()

In [12]:
from sklearn.model_selection import ShuffleSplit
cv = ShuffleSplit(n_splits = 1, test_size = 0.1)

In [13]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
my_classifier = KerasClassifier(make_model, batch_size=256)
# define the grid search parameters
#activation = ['relu', 'elu', 'tanh']
#param_grid = dict(activation=activation)
#param_grid = {'activation' : activation}
validator = GridSearchCV(my_classifier, param_grid={'activation': ['elu'],
                                                   'epochs': [5],
                                                   'filters': [32],
                                                   'Sdroprate': [0.4],
                                                   'droprate': [0.1],
                                                   'layers': [2, 3, 4, 5, 6]},
                                                   cv = cv, 
                                                   scoring = 'roc_auc', 
                                                   n_jobs=1)

In [14]:
grid_result = validator.fit(x_train, y_train)

Epoch 1/5
143613/143613 [==============================] - 311s 2ms/step - loss: 0.0896 - acc: 0.9736
Epoch 2/5
143613/143613 [==============================] - 309s 2ms/step - loss: 0.0554 - acc: 0.9804
Epoch 3/5
143613/143613 [==============================] - 309s 2ms/step - loss: 0.0515 - acc: 0.9811
Epoch 4/5
143613/143613 [==============================] - 308s 2ms/step - loss: 0.0493 - acc: 0.9817
Epoch 5/5
143613/143613 [==============================] - 307s 2ms/step - loss: 0.0477 - acc: 0.9822
Epoch 1/5
143613/143613 [==============================] - 411s 3ms/step - loss: 0.0848 - acc: 0.9744
Epoch 2/5
143613/143613 [==============================] - 410s 3ms/step - loss: 0.0532 - acc: 0.9807
Epoch 3/5
143613/143613 [==============================] - 409s 3ms/step - loss: 0.0493 - acc: 0.9817
Epoch 4/5
143613/143613 [==============================] - 409s 3ms/step - loss: 0.0471 - acc: 0.9822
Epoch 5/5
143613/143613 [==============================] - 409s 3ms/step - loss: 0

In [15]:
print('The parameters of the best model are: ')
print(grid_result.best_params_)
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
# validator.best_estimator_ returns sklearn-wrapped version of best model.
# validator.best_estimator_.model returns the (unwrapped) keras model
best_model = validator.best_estimator_.model
metric_names = best_model.metrics_names
metric_values = best_model.evaluate(x_test, y_test)
for metric, value in zip(metric_names, metric_values):
    print(metric, ': ', value)

The parameters of the best model are: 
{'Sdroprate': 0.4, 'activation': 'elu', 'droprate': 0.1, 'epochs': 5, 'filters': 32, 'layers': 4}
0.986281 (0.000000) with: {'Sdroprate': 0.4, 'activation': 'elu', 'droprate': 0.1, 'epochs': 5, 'filters': 32, 'layers': 2}
0.987916 (0.000000) with: {'Sdroprate': 0.4, 'activation': 'elu', 'droprate': 0.1, 'epochs': 5, 'filters': 32, 'layers': 3}
0.988547 (0.000000) with: {'Sdroprate': 0.4, 'activation': 'elu', 'droprate': 0.1, 'epochs': 5, 'filters': 32, 'layers': 4}
0.988392 (0.000000) with: {'Sdroprate': 0.4, 'activation': 'elu', 'droprate': 0.1, 'epochs': 5, 'filters': 32, 'layers': 5}
0.988314 (0.000000) with: {'Sdroprate': 0.4, 'activation': 'elu', 'droprate': 0.1, 'epochs': 5, 'filters': 32, 'layers': 6}
63978/63978 [==============================] - 64s 1ms/step
loss :  0.0690176298667675
acc :  0.971669946969284


In [16]:
y_pred = best_model.predict(x_test, batch_size=256)

In [17]:
score = roc_auc_score(y_test, y_pred)
print("\n roc_auc score: %.6f \n" % (score))


 roc_auc score: 0.981736 

